In [1]:
# import importlib
# import local_utils; importlib.reload(local_utils)
from local_utils import *

/opt/conda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Seed: 7961730


In [2]:
# Load Data
ids, comments, Y, test_ids, test_comments, inx2label, label2inx = load_data(True)
Y_wblank = np.concatenate([Y, np.expand_dims((~Y.any(axis=1)).astype(int), 1)], axis=1)
print("Original:\n" + comments[0])
print()

comments = Parallel(n_jobs=cpu_cores)(delayed(preprocess)(text, True) for text in comments)
test_comments = Parallel(n_jobs=cpu_cores)(delayed(preprocess)(text, True) for text in test_comments)
print("Processed:\n" + comments[0])

Original:
explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since I'm retired now. 

Processed:
explanation why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now. 


In [3]:
# bad_comments = np.extract(Y.sum(axis=1) > 0, comments)

In [4]:
# # SentencePiece

# with open('bpe/train.txt', 'w', encoding='utf-8') as file:
#     for i in range(0, len(comments)):
#         file.write(comments[i]+'\n')

# with open('bpe/test.txt', 'w', encoding='utf-8') as file:
#     for i in range(0, len(test_comments)):
#         file.write(test_comments[i]+'\n')

# spm_train --input=train.txt,test.txt --model_prefix=toxic --vocab_size=500 --model_type=unigram --num_threads 8
# spm_export_vocab --model=toxic.model --output=toxic.vocab.txt

# spm_encode --model=toxic.model --output_format=piece < train.txt > train_piece.txt
# spm_encode --model=toxic.model --output_format=piece < test.txt > test_piece.txt

# spm_encode --model=toxic.model --output_format=id < train.txt > train_id.txt
# spm_encode --model=toxic.model --output_format=id < test.txt > test_id.txt

# with open('bpe/train_piece.txt', 'r', encoding='utf-8') as file:
#     piece_seq = [line.rstrip('\n') for line in file]
        
# with open('bpe/test_piece.txt', 'r', encoding='utf-8') as file:
#     test_piece_seq = [line.rstrip('\n') for line in file]

# with open('bpe/train_id.txt', 'r', encoding='utf-8') as file:
#     id_seq = []
#     for line in file:
#         id_seq.append([int(i) for i in line.rstrip('\n').split(' ')])    
    
# with open('bpe/test_id.txt', 'r', encoding='utf-8') as file:
#     test_id_seq = []
#     for line in file:
#         test_id_seq.append([int(i) for i in line.rstrip('\n').split(' ')])

In [5]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("bpe/toxic.model")

def encode_ids(text):
    return np.array(sp.EncodeAsIds(text))+1 # leaving 0 for padding

def encode_pieces(text):
    return sp.EncodeAsPieces(text)

id_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_ids)(comment) for comment in comments)
test_id_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_ids)(comment) for comment in test_comments)

piece_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_pieces)(comment) for comment in comments[:100])
# test_piece_seq = Parallel(n_jobs=cpu_cores)(delayed(encode_pieces)(comment) for comment in test_comments)

In [6]:
doc_len = []
doc_ulen = []
for seq in id_seq:
    doc_len.append(len(seq))
    doc_ulen.append(len(set(seq)))
    
for seq in test_id_seq:
    doc_len.append(len(seq))
    doc_ulen.append(len(set(seq)))
    
meta = np.stack([np.array(doc_len), np.array(doc_ulen)], axis=-1)
meta_mean = meta.mean(axis=0)
meta_std = meta.std(axis=0)
meta = (meta - meta_mean)/meta_std

print(meta_mean)
print(meta_std)
print(meta_mean[0]+2*meta_std[0])
print(meta_mean[0]+3*meta_std[0])

X_meta = meta[:len(comments)]
test_X_meta = meta[len(comments):]

[147.37829472  66.31955489]
[249.71726691  53.14835103]
646.8128285488364
896.5300954620701


In [7]:
max_len = 900
# max_len = int(meta_mean[0]+3*meta_std[0])

# PAD_TOKEN = "_PAD_"
# pad_vec = np.zeros((1, emb_size))

# from gensim.models import KeyedVectors
# model = KeyedVectors.load_word2vec_format("bpe/bpemb/en.wiki.bpe.op"+str(num_op)+".d"+str(emb_size)+".w2v.bin", binary=True)
# emb_vectors = np.concatenate([pad_vec, model.syn0])
# print(emb_vectors.shape)

X = pad_sequences(id_seq, max_len)
test_X = pad_sequences(test_id_seq, max_len)

In [8]:
# Train/Valid splitting
trn_inx, val_inx = stratified_sampling(Y, 0.1, seed)

print("train: {}, valid: {}".format(len(trn_inx), len(val_inx)))
# plot_stratified_sampling(Y, trn_inx, val_inx, inx2label)

train: 143613, valid: 15958


In [9]:
# Model description
model_name = 'bpe_500_model'
def getExpModel(num_words, emb_size, input_shape=X.shape[1], classes=Y.shape[1]):

    x_input = Input(shape=(input_shape,))
    
    emb = Embedding(num_words, emb_size, trainable=True, input_length=input_shape, 
                    embeddings_regularizer=l1_l2(l1=1e-20),
                    name='embs')(x_input)
#     emb = SpatialDropout1D(0.25)(emb)
        
#     rnn = Bidirectional(CuDNNGRU(32, return_sequences=True))(emb)
    
    cnn1 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(emb)
    cnn2 = Conv1D(filters=128, kernel_size=4, activation='relu', padding='same')(emb)
    cnn3 = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(emb)
    
    x = concatenate([emb, cnn1, cnn2, cnn3])
    x = AttentionWeightedAverage()(x)
#     x = Dropout(0.25)(x)

    x_output = Dense(classes, activation='sigmoid')(x)
    return Model(inputs=x_input, outputs=x_output)

model = getExpModel(num_words=500, emb_size=128)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 900)          0                                            
__________________________________________________________________________________________________
embs (Embedding)                (None, 900, 128)     64000       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 900, 128)     49280       embs[0][0]                       
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 900, 128)     65664       embs[0][0]                       
__________________________________________________________________________________________________
conv1d_3 (

In [10]:
model_checkpoint = ModelCheckpoint(models_dir+model_name+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=1, mode='auto')
lr_schedule = LearningRateScheduler(lr_change, verbose=1)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=1, min_lr=0.0001, verbose=1)
# tensorboard = TensorBoard(log_dir='logs', write_graph=False)

In [11]:
batch_size = 256
val_batch_size = 1024
weights = getClassWeights(Y, mu=0.5)

# trn_seq = StratifiedFeatureSequence(X[trn_inx], Y[trn_inx], batch_size)
# trn_seq = FeatureSequence(trn_X, trn_X_meta, trn_Y, batch_size, shuffle=True)
trn_seq = FeatureSequence(X[trn_inx], Y[trn_inx], batch_size, shuffle=True)
val_seq = FeatureSequence(X[val_inx], Y[val_inx], val_batch_size)
roc_auc_eval = RocAucEvaluation(X[val_inx], Y[val_inx], batch_size=val_batch_size)

In [12]:
epochs=50
clr = CyclicLR(base_lr=0.001, max_lr=0.003, step_size=len(trn_seq), mode='triangular2')
model.compile(loss="binary_crossentropy", optimizer=optimizers.Nadam(0.001))
# model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(0.003))
# model.compile(loss=art_loss, optimizer=optimizers.Nadam())

model.fit_generator(
    generator=trn_seq, steps_per_epoch=len(trn_seq),
    validation_data=val_seq, validation_steps=len(val_seq),
    initial_epoch=0, epochs=epochs, shuffle=False, verbose=1,
#     callbacks=[model_checkpoint, clr, early_stop, roc_auc_eval],
    callbacks=[model_checkpoint, lr_reduce, early_stop, roc_auc_eval],
    use_multiprocessing=False, workers=cpu_cores, max_queue_size=8*cpu_cores)

Epoch 1/50
561/561 [==============================] - 55s 98ms/step - loss: 0.1710 - val_loss: 0.1391

Epoch 00001: val_loss improved from inf to 0.13911, saving model to models/bpe_500_model.h5
ROC-AUC: 0.37293964


Epoch 2/50
561/561 [==============================] - 54s 96ms/step - loss: 0.1418 - val_loss: 0.1380

Epoch 00002: val_loss improved from 0.13911 to 0.13803, saving model to models/bpe_500_model.h5
ROC-AUC: 0.51106823


Epoch 3/50
561/561 [==============================] - 54s 96ms/step - loss: 0.3442 - val_loss: 14.4751

Epoch 00003: val_loss did not improve
ROC-AUC: 0.90783814


Epoch 4/50
561/561 [==============================] - 53s 95ms/step - loss: 1552.2986 - val_loss: 4338.0865

Epoch 00004: val_loss did not improve

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
ROC-AUC: 0.50341743


Epoch 5/50
561/561 [==============================] - 53s 95ms/step - loss: 4319.9593 - val_loss: 4304.5794

Epoch 00005: val_loss did not improve



In [13]:
stop

NameError: name 'stop' is not defined

In [ ]:
del model
model = load_model(models_dir+model_name+'.h5', compile=True, 
                   custom_objects={'Attention':Attention, 'AttentionWeightedAverage':AttentionWeightedAverage, 'art_loss':art_loss})

In [ ]:
Y_trn_pred = model.predict(X[trn_inx], batch_size=1024, verbose=0)
losses = compute_losses(Y[trn_inx], Y_trn_pred, eps=1e-5)

trn_loss = sum(losses)/len(losses)
trn_auc = metrics.roc_auc_score(Y[trn_inx], Y_trn_pred)

print()
print("avg_loss: {}".format(trn_loss))
print("ROC AUC: {}".format(trn_auc))

In [ ]:
Y_val_pred = model.predict(X[val_inx], batch_size=1024, verbose=0)
losses = compute_losses(Y[val_inx], Y_val_pred, eps=1e-5)

val_loss = sum(losses)/len(losses)
val_auc = metrics.roc_auc_score(Y[val_inx], Y_val_pred)

print()
print("avg_loss: {}".format(val_loss))
print("ROC AUC: {}".format(val_auc))

In [ ]:
stop

In [ ]:
submission_name = 'bpe_submission_'+str(round(val_loss, 5))+'_'+str(round(val_auc, 5))+'.csv'

sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')
test_pred = model.predict(test_X, batch_size=1024, verbose=1)
sample_submission[inx2label] = test_pred
sample_submission.to_csv(results_dir+submission_name, index=False)

FileLink(results_dir+submission_name)

In [ ]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument, KeyedVectors

# def build_tagged_docs(docs):
#     return [TaggedDocument(tokens, [tag]) for tag, tokens in enumerate(docs)]

# d2v_corpus = build_tagged_docs(text_analyzer.transform_to_tokens(piece_seq)+text_analyzer.transform_to_tokens(test_piece_seq))
# print(len(d2v_corpus))

# d2v_model = Doc2Vec(dm=0, dbow_words=1, size=300, window=12, min_count=0, iter=30, workers=cpu_cores)

# %time d2v_model.build_vocab(d2v_corpus)
# print(len(d2v_model.wv.vocab))

# %time d2v_model.train(d2v_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.iter)
# d2v_model.save('bpe/bpe_d2v_model')

# from gensim.models.doc2vec import Doc2Vec
# d2v_model = Doc2Vec.load('bpe/bpe_d2v_model')

# emb_matrix = [np.zeros(300)]
# for w in text_analyzer.emb_words[1:]:
#     emb_matrix.append(d2v_model.wv[w])
# #     emb_matrix.append(d2v_model.wv[w]/np.linalg.norm(d2v_model.wv[w]))
    
# emb_matrix = np.stack(emb_matrix)